In [3]:
import c3d
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib

In [15]:
data = []
with open('DSKPre03.c3d', 'rb') as handle:
    reader = c3d.Reader(handle)
    for i, points, analog in reader.read_frames():
        #print('frame {}: point {}, analog {}'.format(i, points.shape, analog.shape))
        data.append(points)
        
data_calibration = []
with open('SubjectionCalibration01.c3d', 'rb') as handle:
    reader_calibration = c3d.Reader(handle)
    for i, points, analog in reader_calibration.read_frames():
        #print('frame {}: point {}, analog {}'.format(i, points.shape, analog.shape))
        data_calibration.append(points)

data = np.array(data)
data_calibration = np.array(data_calibration)

def data2df(data, c3d_reader):
    point_labels = [i.strip() for i in np.array(c3d_reader.point_labels)]

    point_labels_xyz = [i+'_x' for i in point_labels]
    [point_labels_xyz.append(i+'_y') for i in point_labels]
    [point_labels_xyz.append(i+'_z') for i in point_labels]

    data_xyz = data[:,:,0] # x-coordinates,
    data_xyz = np.append(data_xyz, data[:,:,1], axis = 1) # y-coordinates,
    data_xyz = np.append(data_xyz, data[:,:,2], axis = 1) # z-coordinates,
    return pd.DataFrame(data=data_xyz, columns=point_labels_xyz)

df = data2df(data, reader)
df_calibration = data2df(data_calibration, reader_calibration)
df_calibration

Index(['LFHD_x', 'RFHD_x', 'LBHD_x', 'RBHD_x', 'C7_x', 'T10_x', 'CLAV_x',
       'STRN_x', 'RBAK_x', 'LSHO_x',
       ...
       'RTIB_z', 'RANK_z', 'RHEE_z', 'RTOE_z', 'RELB_z', 'RFRM_z', '*39_z',
       '*40_z', '*41_z', '*42_z'],
      dtype='object', length=129)

In [19]:
list(df_calibration.columns)

['LFHD_x',
 'RFHD_x',
 'LBHD_x',
 'RBHD_x',
 'C7_x',
 'T10_x',
 'CLAV_x',
 'STRN_x',
 'RBAK_x',
 'LSHO_x',
 'LUPA_x',
 'LELB_x',
 'LFRM_x',
 'LWRA_x',
 'LWRB_x',
 'LFIN_x',
 'RSHO_x',
 'RUPA_x',
 'RWRA_x',
 'RWRB_x',
 'RFIN_x',
 'LASI_x',
 'RASI_x',
 'LPSI_x',
 'RPSI_x',
 'LTHI_x',
 'LKNE_x',
 'LTIB_x',
 'LANK_x',
 'LHEE_x',
 'LTOE_x',
 'RTHI_x',
 'RKNE_x',
 'RTIB_x',
 'RANK_x',
 'RHEE_x',
 'RTOE_x',
 'RELB_x',
 'RFRM_x',
 '*39_x',
 '*40_x',
 '*41_x',
 '*42_x',
 'LFHD_y',
 'RFHD_y',
 'LBHD_y',
 'RBHD_y',
 'C7_y',
 'T10_y',
 'CLAV_y',
 'STRN_y',
 'RBAK_y',
 'LSHO_y',
 'LUPA_y',
 'LELB_y',
 'LFRM_y',
 'LWRA_y',
 'LWRB_y',
 'LFIN_y',
 'RSHO_y',
 'RUPA_y',
 'RWRA_y',
 'RWRB_y',
 'RFIN_y',
 'LASI_y',
 'RASI_y',
 'LPSI_y',
 'RPSI_y',
 'LTHI_y',
 'LKNE_y',
 'LTIB_y',
 'LANK_y',
 'LHEE_y',
 'LTOE_y',
 'RTHI_y',
 'RKNE_y',
 'RTIB_y',
 'RANK_y',
 'RHEE_y',
 'RTOE_y',
 'RELB_y',
 'RFRM_y',
 '*39_y',
 '*40_y',
 '*41_y',
 '*42_y',
 'LFHD_z',
 'RFHD_z',
 'LBHD_z',
 'RBHD_z',
 'C7_z',
 'T10_z',
 'CLA

# KHK

In [43]:
participant = df.columns[-1].split(':')[0] # name of participant
# determine kicking leg by highests z-value
kick_height_left = max(df[f"{participant}:LKNE_z"])
kick_height_right = max(df[f"{participant}:RKNE_z"])
kicking_leg = 'L' if kick_height_left > kick_height_right else 'R'
print(f"The kicking leg of {participant} was the {kicking_leg} side")
df['KHK'] = df[f"{participant}:{kicking_leg}KNE_z"]
px.line(df['KHK'])

The kicking leg of AlexS was the R side


# DKS

In [45]:
knee_marker = [f"{participant}:{kicking_leg}KNE_{i}" for i in ['x', 'y', 'z']]
shoulder_marker = [f"{participant}:{kicking_leg}SHO_{i}" for i in ['x', 'y', 'z']]
df['DKS'] = df.apply(lambda x: np.linalg.norm(x[knee_marker].values - x[shoulder_marker].values), axis=1)
px.line(df['DKS'])

# Balance_ind

In [61]:
"""Horizontal (x) Distance between the COM and the closest Supporting leg marker of Toe and Heel.
If COM is between these 2 markers the result is 0 (see sequential action paper)

Normalized by COM height in straight stance (SubjectCalibration01 trial)"""

# first check whether left of right foot is closer (abs  of COM vs both feet)

df['distance_ComFoot_L'] = np.sqrt((df[f'{participant}:LTOE_x'] - df[f'{participant}:CentreOfMass_x']) ** 2 + (df[f'{participant}:LTOE_y'] - df[f'{participant}:CentreOfMass_y']) ** 2)
df['distance_ComFoot_R'] = np.sqrt((df[f'{participant}:RTOE_x'] - df[f'{participant}:CentreOfMass_x']) ** 2 + (df[f'{participant}:RTOE_y'] - df[f'{participant}:CentreOfMass_y']) ** 2)
df['closestFoot_COM'] = pd.DataFrame(df['distance_ComFoot_L'] > df['distance_ComFoot_R']).replace(False, 'L').replace(True, 'R')

df['Balance_ind'] = 99999
# now, if COM_x is between the two x values of the closer foot markers then 0 - else take difference
for index, row in df.iterrows():
    closer_side = row['closestFoot_COM']
    if row[f'{participant}:{closer_side}TOE_x'] <= row[f'{participant}:CentreOfMass_x'] <= row[f'{participant}:{closer_side}HEE_x']:
        balance_ind = 0
    elif row[f'{participant}:{closer_side}HEE_x'] <= row[f'{participant}:CentreOfMass_x'] <= row[f'{participant}:{closer_side}TOE_x']:
        balance_ind = 0
    else:
        balance_ind = min(abs(row[f'{participant}:CentreOfMass_x'] - row[f'{participant}:{closer_side}TOE_x']),
                                 abs(row[f'{participant}:CentreOfMass_x'] - row[f'{participant}:{closer_side}HEE_x']))
    df.at[index,'Balance_ind'] = balance_ind


# TODO: normalize by COM height in straight stance!
print('the closer foot analysis checks the toes only.. include the heel as well!')
print('normalize by COM height in straight stance!')

px.line(df['Balance_ind'])

the closer foot analysis checks the toes only.. include the heel as well!
normalize by COM height in straight stance!


# Stability ind.

In [7]:
"""Absolute distance between the COM and the closest supporting leg marker"""
print('which leg markers?!')

which leg markers?!


In [23]:
[col for col in df.columns if 'HJC' in col]

['AlexS:LHJC_x',
 'AlexS:RHJC_x',
 'AlexS:LHJC_y',
 'AlexS:RHJC_y',
 'AlexS:LHJC_z',
 'AlexS:RHJC_z']